In [3]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((20,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54063
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54064
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54065
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54066
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54067
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54068
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54069
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54070
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54071
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54072
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54073
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54074
Train: 54000 Val:6000 Test:10000
INFO:ro

INFO:root:(('127.0.0.1', 54076)) Conexión aceptada con ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54076)) Conexión aceptada con ('127.0.0.1', 54063)
INFO:root:(('127.0.0.1', 54076)) Conexión aceptada con ('127.0.0.1', 54069)
INFO:root:(('127.0.0.1', 54076)) Conexión aceptada con ('127.0.0.1', 54066)
INFO:root:(('127.0.0.1', 54076)) Conexión aceptada con ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54075)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54074)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54072)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54071)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54073)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54070)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54068)
INFO:root:(('127.0.0.1', 54077)) Conexión aceptada con ('127.0.0.1', 54064)
INFO:root:((

INFO:root:(('127.0.0.1', 54070)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54072)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54071)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54073)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54075)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54074)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54076)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54077)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54078)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54079)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54063)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 54080)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54081)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54082)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54063)) Broadcasting model to all clients (19)...
INFO:root:(('127.0.0.1', 54064)) Model initialized
INFO:root:(('127.0.0.1', 54065)

INFO:root:(('127.0.0.1', 54071)) Model added (2/20)
INFO:root:(('127.0.0.1', 54068)) Model added (2/20)
INFO:root:(('127.0.0.1', 54064)) Model added (2/20)
INFO:root:(('127.0.0.1', 54069)) Model added (2/20)
INFO:root:(('127.0.0.1', 54067)) Model added (2/20)
INFO:root:(('127.0.0.1', 54065)) Model added (2/20)
INFO:root:(('127.0.0.1', 54066)) Model added (2/20)
INFO:root:(('127.0.0.1', 54074)) Model added (2/20)
INFO:root:(('127.0.0.1', 54063)) Model added (2/20)
INFO:root:(('127.0.0.1', 54079)) Model added (2/20)
INFO:root:(('127.0.0.1', 54075)) Model added (2/20)
INFO:root:(('127.0.0.1', 54078)) Model added (2/20)
INFO:root:(('127.0.0.1', 54064)) RECIVING MODEL! from ('127.0.0.1', 54063)
INFO:root:(('127.0.0.1', 54065)) RECIVING MODEL! from ('127.0.0.1', 54063)
INFO:root:(('127.0.0.1', 54066)) RECIVING MODEL! from ('127.0.0.1', 54063)
INFO:root:(('127.0.0.1', 54067)) RECIVING MODEL! from ('127.0.0.1', 54063)
INFO:root:(('127.0.0.1', 54068)) RECIVING MODEL! from ('127.0.0.1', 54063)
I

INFO:root:(('127.0.0.1', 54069)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54066)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54079)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54078)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54080)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54081)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54082)) RECIVING MODEL! from ('127.0.0.1', 54077)
INFO:root:(('127.0.0.1', 54066)) RECIVING MODEL! from ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54065)) RECIVING MODEL! from ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54080)) Model added (3/20)
INFO:root:(('127.0.0.1', 54063)) RECIVING MODEL! from ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54064)) RECIVING MODEL! from ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54068)) RECIVING MODEL! from ('127.0.0.1', 54067)
INFO:root:(('127.0.0.1', 54069)) RECIVING MODEL!

...

INFO:root:(('127.0.0.1', 54073)) Model added (5/20)
INFO:root:(('127.0.0.1', 54072)) Model added (5/20)
INFO:root:(('127.0.0.1', 54074)) Model added (5/20)
INFO:root:(('127.0.0.1', 54076)) Model added (5/20)
INFO:root:(('127.0.0.1', 54075)) Model added (5/20)
INFO:root:(('127.0.0.1', 54077)) Model added (5/20)
INFO:root:(('127.0.0.1', 54080)) Model added (4/20)
INFO:root:(('127.0.0.1', 54079)) Model added (5/20)
INFO:root:(('127.0.0.1', 54078)) Model added (5/20)
INFO:root:(('127.0.0.1', 54082)) Model added (5/20)
INFO:root:(('127.0.0.1', 54081)) Model added (5/20)
INFO:root:(('127.0.0.1', 54067)) Model added (5/20)
INFO:root:(('127.0.0.1', 54063)) Model added (6/20)
INFO:root:(('127.0.0.1', 54066)) Model added (6/20)
INFO:root:(('127.0.0.1', 54064)) Model added (6/20)
INFO:root:(('127.0.0.1', 54065)) Model added (6/20)
INFO:root:(('127.0.0.1', 54068)) Model added (6/20)
INFO:root:(('127.0.0.1', 54071)) Model added (6/20)
INFO:root:(('127.0.0.1', 54069)) Model added (6/20)
INFO:root:((

INFO:root:(('127.0.0.1', 54071)) Model added (13/20)
INFO:root:(('127.0.0.1', 54069)) Model added (13/20)
INFO:root:(('127.0.0.1', 54070)) Model added (13/20)
INFO:root:(('127.0.0.1', 54073)) Model added (13/20)
INFO:root:(('127.0.0.1', 54072)) Model added (13/20)
INFO:root:(('127.0.0.1', 54074)) Model added (13/20)
INFO:root:(('127.0.0.1', 54076)) Model added (13/20)
INFO:root:(('127.0.0.1', 54075)) Model added (13/20)
INFO:root:(('127.0.0.1', 54077)) Model added (13/20)
INFO:root:(('127.0.0.1', 54080)) Model added (12/20)
INFO:root:(('127.0.0.1', 54079)) Model added (13/20)
INFO:root:(('127.0.0.1', 54078)) Model added (13/20)
INFO:root:(('127.0.0.1', 54082)) Model added (13/20)
INFO:root:(('127.0.0.1', 54081)) Model added (13/20)
INFO:root:(('127.0.0.1', 54067)) Model added (13/20)
INFO:root:(('127.0.0.1', 54063)) Model added (14/20)
INFO:root:(('127.0.0.1', 54066)) Model added (14/20)
INFO:root:(('127.0.0.1', 54064)) Model added (14/20)
INFO:root:(('127.0.0.1', 54065)) Model added (

INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:(('127.0.0.1', 54080)) Model added (20/20)
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:(('127.0.0.1', 54065)) Finish!!.
INFO:root:(('127.0.0.1', 54064)) Finish!!.
INFO:root:(('127.0.0.1', 54063)) Finish!!.
INFO:root:(('127.0.0.1', 54066)) Finish!!.
INFO:root:(('127.0.0.1', 54068)) Finish!!.
INFO:root:(('127.0.0.1', 54071)) Finish!!.
INFO:root:(('127.0.0.1', 54070)) Finish!!.
INFO:root:(('127.0.0.1', 54069)) Finish!!.
INFO:root:(('127.0.0.1'

DEBUG:root:Closed connection: ('127.0.0.1', 54068)
DEBUG:root:Closed connection: ('127.0.0.1', 54071)
DEBUG:root:Closed connection: ('127.0.0.1', 54072)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54074)
DEBUG:root:Closed connection: ('127.0.0.1', 54075)
DEBUG:root:Closed connection: ('127.0.0.1', 54076)
DEBUG:root:Closed connection: ('127.0.0.1', 54077)
DEBUG:root:Closed connection: ('127.0.0.1', 54078)
DEBUG:root:Closed connection: ('127.0.0.1', 54079)
DEBUG:root:Closed connection: ('127.0.0.1', 54080)
DEBUG:root:Closed connection: ('127.0.0.1', 54082)
DEBUG:root:Closed connection: ('127.0.0.1', 54081)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 54067
DEBUG:root:Closed connection: ('127.0.0.1', 54067)
DEBUG:root:Closed connection: ('127.0.0.1', 54067)
DEBUG:root:Closed connection: ('127.0.0.1', 54067)
DEBUG:root:Closed connection: ('127.0.0.1', 54067)
DEBUG:root:Closed connection: ('127.0.0.1', 54067)
DEBUG:root:Close

DEBUG:root:Closed connection: ('127.0.0.1', 54076)
DEBUG:root:Closed connection: ('127.0.0.1', 54079)
DEBUG:root:Closed connection: ('127.0.0.1', 54080)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54077)
DEBUG:root:Closed connection: ('127.0.0.1', 54081)
DEBUG:root:Closed connection: ('127.0.0.1', 54078)
DEBUG:root:Closed connection: ('127.0.0.1', 54082)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 54073
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54074)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Closed connection: ('127.0.0.1', 54073)
DEBUG:root:Close

In [2]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time

@pytest.mark.parametrize('n',[2,4]) 
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=1)

    # Stopping node
    nodes[1].stop()

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()
        
test_node_down_on_learning(5)
    #break

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59310
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59311
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59312
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59313
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 59314
INFO:root:(('127.0.0.1', 59310)) Conexión aceptada con ('127.0.0.1', 59311)
INFO:root:(('127.0.0.1', 59311)) Conexión aceptada con ('127.0.0.1', 59312)
INFO:root:(('127.0.0.1', 59312)) Conexión aceptada con ('127.0.0.1', 59310)
INFO:root:(('127.0.0.1', 59312)) Conexión aceptada con ('127.0.0.1', 59313)
INFO:root:(('127.0.0.1', 59313)) Conexión aceptada con ('127.0.0.1', 59311)
INFO:root:(('127.0.0.1', 59313)) Conexión aceptada con ('127.0.0.1', 59310)
INFO:root:(('127.0.0.1', 59313)) Conexión aceptada con ('127.0.0.1', 59314)
INFO:root:(('127.0.0.1', 59314)) Conexión aceptada con ('127.0.0.1',

GPU available: False, used: False


INFO:root:(('127.0.0.1', 59314)) Model initialized


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


INFO:root:(('127.0.0.1', 59312)) Model initialized


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 59313)) Model added (1/4) from ('127.0.0.1', 59313)
INFO:root:(('127.0.0.1', 59313)) Broadcasting model to 3 clients. (size: 547264 bytes)
INFO:root:(('127.0.0.1', 59312)) Model added (1/4) from ('127.0.0.1', 59313)
INFO:root:(('127.0.0.1', 59314)) Model added (1/4) from ('127.0.0.1', 59313)
INFO:root:(('127.0.0.1', 59310)) Model added (1/4) from ('127.0.0.1', 59313)
INFO:root:(('127.0.0.1', 59312)) Model added (2/4) from ('127.0.0.1', 59312)
INFO:root:(('127.0.0.1', 59312)) Broadcasting model to 3 clients. (size: 547264 bytes)
INFO:root:(('127.0.0.1', 59313)) Model added (2/4) from ('127.0.0.1', 59312)
INFO:root:(('127.0.0.1', 59314)) Model added (2/4) from ('127.0.0.1', 59312)
INFO:root:(('127.0.0.1', 59310)) Model added (2/4) from ('127.0.0.1', 59312)
INFO:root:(('127.0.0.1', 59314)) Model added (3/4) from ('127.0.0.1', 59314)
INFO:root:(('127.0.0.1', 59314)) Broadcasting model to 3 clients. (size: 547264 bytes)
INFO:root:(('127.0.0.1', 59310)) Model added (

In [2]:
for node in nodes:
    for nei in node.neightboors:
        print(nei.get_addr())
        print(len(nei.param_bufffer))
        print(nei.tmp)
        
    print("-------")

('127.0.0.1', 55430)
0
1
('127.0.0.1', 55431)
0
1
-------
-------
('127.0.0.1', 55428)
0
1
('127.0.0.1', 55431)
0
1
-------
('127.0.0.1', 55430)
0
1
('127.0.0.1', 55428)
0
1
-------


In [9]:
len(nodes[0].learner.encode_parameters())

544640